In [ ]:
!pip3 install 'tqdm'  

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import math
from sklearn.preprocessing import normalize
import copy
import torchvision.datasets as dsets
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from torch.utils.data import Subset, DataLoader
import random
from sklearn.metrics import confusion_matrix 
import seaborn as sn
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
if not os.path.isdir('./OpenWorld'):
  !git clone https://github.com/Alessia-Sc/Open-World-Recognition.git OpenWorld

from OpenWorld.CIFAR100 import DatasetCifar100
from OpenWorld.ResNet import resnet32
from tqdm.notebook import tqdm

Cloning into 'OpenWorld'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 147 (delta 86), reused 19 (delta 10), pack-reused 0
Receiving objects: 100% (147/147), 36.63 KiB | 1.05 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
DEVICE="cuda"
TOTAL_CLASSES = 50
NUM_CLASSES = 10

BATCH_SIZE=128
LR=2.0
MOMENTUM=0.9
WEIGHT_DECAY=1e-5

NUM_EPOCHS=70
GAMMA=0.2
MILESTONE=[49,63]

K=2000

In [ ]:
class iCaRLNet(): 
    def __init__(self, feature_size, n_classes, lr=LR, momentum=MOMENTUM, gamma=GAMMA, weight_decay=WEIGHT_DECAY, milestone=MILESTONE, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS):
        # Network architecture
        super(iCaRLNet, self).__init__()
        self.net = resnet32(num_classes=n_classes)
        self.forward = self.net.forward
        self.lr = lr
        self.gamma = gamma
        self.weight_decay = weight_decay
        self.milestone = milestone
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.n_classes = 0
        self.n_known = 0
        self.feature_size=feature_size
        self.momentum=momentum
        self.feature_extractor = self.net.get_feat_ext

        
        self.loss = nn.BCEWithLogitsLoss()

        self.compute_means = True
        self.exemplar_means = []
        self.exemplar_sets = []
        


    def construct_exemplar_set(self, images, m):
        
        features = np.zeros((0,self.feature_size))
        indices = np.zeros((0), dtype=int)
        dl = DataLoader(images, batch_size=self.batch_size,shuffle=False, num_workers=4)
        with torch.no_grad():
          for ind, img, lab in dl:
            x = Variable(img).cuda()
            feature = self.feature_extractor(x).data.cpu().numpy()
            feature = normalize(feature, axis=1, norm='l2')   
            features = np.concatenate((features,feature), axis=0)  
            indices = np.concatenate((indices,ind), axis=0)

        class_mean = np.mean(features, axis=0)
        class_mean = class_mean / np.linalg.norm(class_mean)  # Normalize

        exemplar_set = []
        exemplar_features = np.zeros((0,64))

        for k in range(1, int(m)+1):
            S = np.sum(exemplar_features, axis=0)
            phi = features    
            mu = class_mean 
            mu_p = 1.0 / k * (phi + S)
            mu_p = normalize(mu_p, axis=1, norm='l2')
            i = np.argmin(np.sqrt(np.sum((mu - mu_p) ** 2, axis=1)))
            exemplar_set.append(indices[i])  
            addfeature =  np.expand_dims(features[i], axis=0)
            exemplar_features = np.concatenate((exemplar_features,addfeature), axis=0)

            features = np.delete(features, i, 0)
            indices = np.delete(indices, i, 0)
            
        self.exemplar_sets.append(exemplar_set) 
        

    def reduce_exemplar_sets(self, m):
        for y, P_y in enumerate(self.exemplar_sets):
            self.exemplar_sets[y] = P_y[:int(m)]


    def exemplarIndexes(self):
        Indexes = []
        for P_y in self.exemplar_sets:
            Indexes += P_y
        return Indexes


    def update_representation(self, cifar, batchindexes):

        prev_model = copy.deepcopy(self)
        prev_model = prev_model.net.eval().cuda()
        self.net = self.net.cuda()
        self.compute_means = True

        self.n_classes += 10

        # Form combined training set
        newindexes = []
        if self.n_classes > 10:    
            newindexes = self.exemplarIndexes() 
        newindexes += list(batchindexes)
        
        reprdata = Subset(cifar, newindexes)

        loader = DataLoader(reprdata, batch_size=self.batch_size,shuffle=True, num_workers=4, drop_last=True)

        optimizer = optim.SGD(self.net.parameters(), lr=self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.milestone, gamma=self.gamma)

        for epoch in tqdm(range(self.num_epochs)):
            losses = []
            for _, images, labels in loader:
                labels = torch.tensor([torch.tensor(train_ds.dict_class_label[c.item()]) for c in labels])
                images = Variable(torch.FloatTensor(images)).cuda()
                labels = Variable(labels).cuda()
                optimizer.zero_grad()
                g = self.forward(images)
                
                y_hot = F.one_hot(labels, self.n_classes).float().cuda()
                
                if self.n_known > 0:
                    q = prev_model.forward(images)
                    q = torch.sigmoid(q)
                    target = torch.cat((q[:,:self.n_known], y_hot[:,self.n_known:self.n_classes]), dim=1) 
                    loss = self.loss(g[:,:self.n_classes], target)  
                else:
                    loss = self.loss(g[:,:self.n_classes], y_hot[:,:self.n_classes]) 

                losses.append(loss.item())
                loss.backward()
                optimizer.step()
            
            scheduler.step()    
        

In [ ]:
train_ds = DatasetCifar100(split="train")
test_ds = DatasetCifar100(split='test')

closedWorldclasses = train_ds.closedWClasses
openWorldclasses = train_ds.openWClasses


Extracting ./CIFAR_100/cifar-100-python.tar.gz to ./CIFAR_100
Files already downloaded and verified


In [ ]:
def accuracy_with_reject(dl, icarl, cont):
    total = 0.0
    correct = 0.0
    all_rej = 0.0
    if cont==1:
      tau=0.55
    if cont==2:
      tau=0.55
    if cont==3:
      tau=0.58
    if cont==4:
      tau=0.62
    if cont==5:
      tau=0.66
    
    for _, images, labels in dl: 
        images = Variable(images).cuda()
        out = icarl.forward(images)
        
        probs, preds = torch.max(torch.softmax(out, dim=1), dim=1, keepdim=False)
        new_preds = preds
        rejected = np.ones(len(images))


        for i in range(128):    #Iterate over images
          if probs[i] > tau:
              rejected[i] = 0

        for i in range(len(rejected)):
          if rejected[i]==1:                      #Rejected
            if labels[i] in closedWorldclasses:    #Wrong at rejecting
                new_preds[i]=-1         
            else:
                new_preds[i] = train_ds.dict_class_label[labels[i].item()]        #Right in rejecting
  
          elif rejected[i]==0:                # Not rejected
            if labels[i].item() in openWorldclasses: 
              new_preds[i] = -1                 #Wrong in not rejecting
              

        all_rej += rejected.sum()
         
        labels = torch.tensor([torch.tensor(train_ds.dict_class_label[c.item()]) for c in labels])
        total = total + len(labels)
        correct += (new_preds.data.cpu() == labels.cpu()).sum()

    acc = 100 * correct / total
    return acc, all_rej, total

## Tutti i World with Rejection

In [ ]:
torch.cuda.current_device()
torch.cuda._initialized = True

icarl = iCaRLNet(64, TOTAL_CLASSES)
icarl.net = icarl.net.to(DEVICE)

acc_vect = []
cont=0
for s in range(5):
    cont+=1
    print("\n")
    print('-' * 80)
    print(f"ITERATION: {(s+1)*10}/50")

    print("Loading training examples for classes:", closedWorldclasses[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES])
    batchindexes = train_ds.get_indexes_from_labels(closedWorldclasses[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES])
    batch = Subset(train_ds, batchindexes)

    print("Loading Closed test examples for classes:", closedWorldclasses[0:s*NUM_CLASSES + NUM_CLASSES])
    test_closedIdx = test_ds.get_indexes_from_labels(closedWorldclasses[0:s*NUM_CLASSES + NUM_CLASSES])
    test_closed =  Subset(test_ds, test_closedIdx)
    
    print("Loading Open test examples for classes:", openWorldclasses[0:s*NUM_CLASSES + NUM_CLASSES]) 
    test_openIdx = test_ds.get_indexes_from_labels(openWorldclasses[0:s*NUM_CLASSES + NUM_CLASSES])
    test_open = Subset(test_ds, test_openIdx)

    print("\nBatch size train: {} - Batch size test: {}".format(len(batch), len(test_open)))

    train_loader = DataLoader(batch, batch_size=BATCH_SIZE,shuffle=True, num_workers=4, drop_last=True)
    test_loader_open = DataLoader(test_open, batch_size=BATCH_SIZE,shuffle=True, num_workers=4, drop_last=True) 
    test_loader_closed = DataLoader(test_closed, batch_size=BATCH_SIZE,shuffle=True, num_workers=4, drop_last=True) 

    icarl.net.train()
    icarl.update_representation(train_ds, batchindexes)
    icarl.net.eval()
    m = K / icarl.n_classes 

    icarl.reduce_exemplar_sets(m)

    # Construct exemplar sets for new classes
    for y in closedWorldclasses[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES]:
        imagesInd = train_ds.get_indexes_from_labels([y]) 
        images = Subset(train_ds, imagesInd)
        icarl.construct_exemplar_set(images, m) 

    icarl.n_known = icarl.n_classes
 
    acc, rejected, total = accuracy_with_reject(train_loader, icarl, cont)
    print(f'Train Accuracy: {round(acc.item(),2)}')
    
    acc, rejected, total = accuracy_with_reject(test_loader_closed, icarl, cont)
    print(f'\nClosed Test Accuracy: {round(acc.item(),2)}')
    print(f'Number of rejected images: {rejected}')
    print(f'Percentage of rejected images: {round(rejected/total*100,2)} %')
    print(f'Number of samples NotRejSamples/Rejected: {round((total-rejected)/rejected,2)}')
    
    acc, rejected,total = accuracy_with_reject(test_loader_open, icarl, cont)
    print(f'\nOpen Test Accuracy: {round(acc.item(),2)}')
    print(f'Number of rejected images: {rejected}')
    print(f'Percentage of rejected images: {round(rejected/total*100,2)} %')
    print(f'Number of samples NotRejSamples/Rejected: {round((total-rejected)/rejected,2)}')




--------------------------------------------------------------------------------
ITERATION: 10/50
Loading training examples for classes: [36, 61, 49, 58, 92, 90, 68, 32, 28, 52]
Loading Closed test examples for classes: [36, 61, 49, 58, 92, 90, 68, 32, 28, 52]
Loading Open test examples for classes: [30, 2, 64, 71, 19, 35, 31, 63, 54, 15]

Batch size train: 5000 - Batch size test: 1000



Train Accuracy: 77.48

Closed Test Accuracy: 71.76
Number of rejected images: 241.0
Percentage of rejected images: 26.9 %
Number of samples NotRejSamples/Rejected: 2.72

Open Test Accuracy: 76.56
Number of rejected images: 686.0
Percentage of rejected images: 76.56 %
Number of samples NotRejSamples/Rejected: 0.31


--------------------------------------------------------------------------------
ITERATION: 20/50
Loading training examples for classes: [47, 87, 1, 41, 93, 6, 88, 12, 38, 91]
Loading Closed test examples for classes: [36, 61, 49, 58, 92, 90, 68, 32, 28, 52, 47, 87, 1, 41, 93, 6, 88, 12, 38, 91]
Loading Open test examples for classes: [30, 2, 64, 71, 19, 35, 31, 63, 54, 15, 43, 73, 40, 55, 7, 78, 14, 10, 70, 44]

Batch size train: 5000 - Batch size test: 2000



Train Accuracy: 80.27

Closed Test Accuracy: 60.52
Number of rejected images: 637.0
Percentage of rejected images: 33.18 %
Number of samples NotRejSamples/Rejected: 2.01

Open Test Accuracy: 71.98
Number of rejected images: 1382.0
Percentage of rejected images: 71.98 %
Number of samples NotRejSamples/Rejected: 0.39


--------------------------------------------------------------------------------
ITERATION: 30/50
Loading training examples for classes: [81, 33, 8, 48, 60, 27, 50, 17, 56, 97]
Loading Closed test examples for classes: [36, 61, 49, 58, 92, 90, 68, 32, 28, 52, 47, 87, 1, 41, 93, 6, 88, 12, 38, 91, 81, 33, 8, 48, 60, 27, 50, 17, 56, 97]
Loading Open test examples for classes: [30, 2, 64, 71, 19, 35, 31, 63, 54, 15, 43, 73, 40, 55, 7, 78, 14, 10, 70, 44, 0, 86, 79, 57, 75, 46, 83, 82, 22, 4]

Batch size train: 5000 - Batch size test: 3000



Train Accuracy: 84.09

Closed Test Accuracy: 55.2
Number of rejected images: 1088.0
Percentage of rejected images: 36.96 %
Number of samples NotRejSamples/Rejected: 1.71

Open Test Accuracy: 70.52
Number of rejected images: 2076.0
Percentage of rejected images: 70.52 %
Number of samples NotRejSamples/Rejected: 0.42


--------------------------------------------------------------------------------
ITERATION: 40/50
Loading training examples for classes: [34, 42, 84, 66, 62, 26, 29, 51, 3, 72]
Loading Closed test examples for classes: [36, 61, 49, 58, 92, 90, 68, 32, 28, 52, 47, 87, 1, 41, 93, 6, 88, 12, 38, 91, 81, 33, 8, 48, 60, 27, 50, 17, 56, 97, 34, 42, 84, 66, 62, 26, 29, 51, 3, 72]
Loading Open test examples for classes: [30, 2, 64, 71, 19, 35, 31, 63, 54, 15, 43, 73, 40, 55, 7, 78, 14, 10, 70, 44, 0, 86, 79, 57, 75, 46, 83, 82, 22, 4, 45, 18, 89, 5, 59, 21, 95, 96, 69, 16]

Batch size train: 5000 - Batch size test: 4000



Train Accuracy: 77.36

Closed Test Accuracy: 44.66
Number of rejected images: 1824.0
Percentage of rejected images: 45.97 %
Number of samples NotRejSamples/Rejected: 1.18

Open Test Accuracy: 70.69
Number of rejected images: 2805.0
Percentage of rejected images: 70.69 %
Number of samples NotRejSamples/Rejected: 0.41


--------------------------------------------------------------------------------
ITERATION: 50/50
Loading training examples for classes: [39, 9, 37, 85, 13, 25, 11, 67, 99, 74]
Loading Closed test examples for classes: [36, 61, 49, 58, 92, 90, 68, 32, 28, 52, 47, 87, 1, 41, 93, 6, 88, 12, 38, 91, 81, 33, 8, 48, 60, 27, 50, 17, 56, 97, 34, 42, 84, 66, 62, 26, 29, 51, 3, 72, 39, 9, 37, 85, 13, 25, 11, 67, 99, 74]
Loading Open test examples for classes: [30, 2, 64, 71, 19, 35, 31, 63, 54, 15, 43, 73, 40, 55, 7, 78, 14, 10, 70, 44, 0, 86, 79, 57, 75, 46, 83, 82, 22, 4, 45, 18, 89, 5, 59, 21, 95, 96, 69, 16, 98, 23, 80, 65, 76, 77, 20, 24, 94, 53]

Batch size train: 5000 - Ba


Train Accuracy: 87.38

Closed Test Accuracy: 46.69
Number of rejected images: 1721.0
Percentage of rejected images: 34.48 %
Number of samples NotRejSamples/Rejected: 1.9

Open Test Accuracy: 50.44
Number of rejected images: 2518.0
Percentage of rejected images: 50.44 %
Number of samples NotRejSamples/Rejected: 0.98
